In [1]:
from xgboost import XGBClassifier
import pandas as pd
import sklearn.ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, precision_score, matthews_corrcoef, confusion_matrix, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import statistics
import numpy as np

In [2]:
df = pd.read_csv("data/4-featurized_introns_data.tsv", sep="\t")
df

,chr,start,end,strand,class,GTExv2,TCGAv2,SRAv3h,RC3-Splice_site,RC3-Score,...,repeat_features:Low complexity regions,repeat_features:Tandem repeats,repeat_features:Unknown,repeat_features:Satellite repeats,repeat_features:Centromere,repeat_features:RNA repeats,repeat_features:Type II Transposons,repeat_features:LTRs,Acceptor splice site,Donor splice site
0,chr1,12227,12612,+,1,1122:3199,583:1460,9374:54492,GT:AG,59151,...,0,0,0,0,0,0,0,0,GT,AG
1,chr1,12721,13220,+,1,1791:3198,783:1104,14048:56719,GT:AG,61021,...,0,0,0,0,0,0,0,0,GT,AG
2,chr1,12057,12178,+,1,NaN,NaN,NaN,NaN,-1,...,0,0,0,0,0,0,0,0,NaN,NaN
3,chr1,12697,12974,+,1,1:1,1:1,15:19,GT:AG,21,...,0,0,0,0,0,0,0,0,GT,AG
4,chr1,13052,13220,+,1,22:24,17:18,433:484,GC:AG,526,...,0,0,0,0,0,0,0,0,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517420,chrX,151403679,151404937,+,0,NaN,NaN,NaN,NaN,-1,...,0,1,0,0,0,0,0,0,NaN,NaN
517421,chrX,151409210,151456968,+,0,NaN,NaN,NaN,NaN,-1,...,6,22,1,0,0,0,8,17,NaN,NaN
517422,chrX,153906577,153906694,-,0,NaN,NaN,NaN,NaN,-1,...,0,0,0,0,0,0,0,0,NaN,NaN
517423,chrX,153906409,153906520,-,0,NaN,NaN,NaN,NaN,-1,...,0,0,0,0,0,0,0,0,NaN,NaN


In [3]:
df = df.drop(columns=["GTExv2", "TCGAv2", "SRAv3h"])
df

,chr,start,end,strand,class,RC3-Splice_site,RC3-Score,repeat_features:Type I Transposons/LINE,repeat_features:Type I Transposons/SINE,repeat_features:Dust,...,repeat_features:Low complexity regions,repeat_features:Tandem repeats,repeat_features:Unknown,repeat_features:Satellite repeats,repeat_features:Centromere,repeat_features:RNA repeats,repeat_features:Type II Transposons,repeat_features:LTRs,Acceptor splice site,Donor splice site
0,chr1,12227,12612,+,1,GT:AG,59151,0,0,0,...,0,0,0,0,0,0,0,0,GT,AG
1,chr1,12721,13220,+,1,GT:AG,61021,0,0,0,...,0,0,0,0,0,0,0,0,GT,AG
2,chr1,12057,12178,+,1,NaN,-1,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,chr1,12697,12974,+,1,GT:AG,21,0,0,0,...,0,0,0,0,0,0,0,0,GT,AG
4,chr1,13052,13220,+,1,GC:AG,526,0,0,0,...,0,0,0,0,0,0,0,0,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517420,chrX,151403679,151404937,+,0,NaN,-1,0,3,5,...,0,1,0,0,0,0,0,0,NaN,NaN
517421,chrX,151409210,151456968,+,0,NaN,-1,37,19,38,...,6,22,1,0,0,0,8,17,NaN,NaN
517422,chrX,153906577,153906694,-,0,NaN,-1,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
517423,chrX,153906409,153906520,-,0,NaN,-1,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [4]:

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df["RC3-Splice_site"])
df["RC3-Splice_site"] = le.transform(df["RC3-Splice_site"])

df

,chr,start,end,strand,class,RC3-Splice_site,RC3-Score,repeat_features:Type I Transposons/LINE,repeat_features:Type I Transposons/SINE,repeat_features:Dust,...,repeat_features:Low complexity regions,repeat_features:Tandem repeats,repeat_features:Unknown,repeat_features:Satellite repeats,repeat_features:Centromere,repeat_features:RNA repeats,repeat_features:Type II Transposons,repeat_features:LTRs,Acceptor splice site,Donor splice site
0,chr1,12227,12612,+,1,4,59151,0,0,0,...,0,0,0,0,0,0,0,0,GT,AG
1,chr1,12721,13220,+,1,4,61021,0,0,0,...,0,0,0,0,0,0,0,0,GT,AG
2,chr1,12057,12178,+,1,6,-1,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,chr1,12697,12974,+,1,4,21,0,0,0,...,0,0,0,0,0,0,0,0,GT,AG
4,chr1,13052,13220,+,1,3,526,0,0,0,...,0,0,0,0,0,0,0,0,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517420,chrX,151403679,151404937,+,0,6,-1,0,3,5,...,0,1,0,0,0,0,0,0,NaN,NaN
517421,chrX,151409210,151456968,+,0,6,-1,37,19,38,...,6,22,1,0,0,0,8,17,NaN,NaN
517422,chrX,153906577,153906694,-,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
517423,chrX,153906409,153906520,-,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [5]:
le = preprocessing.LabelEncoder()
le.fit(df["Acceptor splice site"])
df["Acceptor splice site"] = le.transform(df["Acceptor splice site"])

le = preprocessing.LabelEncoder()
le.fit(df["Donor splice site"])
df["Donor splice site"] = le.transform(df["Donor splice site"])

df

,chr,start,end,strand,class,RC3-Splice_site,RC3-Score,repeat_features:Type I Transposons/LINE,repeat_features:Type I Transposons/SINE,repeat_features:Dust,...,repeat_features:Low complexity regions,repeat_features:Tandem repeats,repeat_features:Unknown,repeat_features:Satellite repeats,repeat_features:Centromere,repeat_features:RNA repeats,repeat_features:Type II Transposons,repeat_features:LTRs,Acceptor splice site,Donor splice site
0,chr1,12227,12612,+,1,4,59151,0,0,0,...,0,0,0,0,0,0,0,0,3,1
1,chr1,12721,13220,+,1,4,61021,0,0,0,...,0,0,0,0,0,0,0,0,3,1
2,chr1,12057,12178,+,1,6,-1,0,0,0,...,0,0,0,0,0,0,0,0,4,4
3,chr1,12697,12974,+,1,4,21,0,0,0,...,0,0,0,0,0,0,0,0,3,1
4,chr1,13052,13220,+,1,3,526,0,0,0,...,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517420,chrX,151403679,151404937,+,0,6,-1,0,3,5,...,0,1,0,0,0,0,0,0,4,4
517421,chrX,151409210,151456968,+,0,6,-1,37,19,38,...,6,22,1,0,0,0,8,17,4,4
517422,chrX,153906577,153906694,-,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,0,4,4
517423,chrX,153906409,153906520,-,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,0,4,4


In [6]:
df_annot_test_y = df.pop("class")

# Drop the columns that might not be available prior to manual gene annotation or irrelevant to ML
df_annot_test_X_1 = df.drop(["chr", "start", "end", "strand", "RC3-Splice_site"], axis=1)
df_annot_test_y

0         1
1         1
2         1
3         1
4         1
         ..
517420    0
517421    0
517422    0
517423    0
517424    0
Name: class, Length: 517425, dtype: int64

In [7]:
df_annot_test_X_1

,RC3-Score,repeat_features:Type I Transposons/LINE,repeat_features:Type I Transposons/SINE,repeat_features:Dust,repeat_features:Simple repeats,repeat_features:Low complexity regions,repeat_features:Tandem repeats,repeat_features:Unknown,repeat_features:Satellite repeats,repeat_features:Centromere,repeat_features:RNA repeats,repeat_features:Type II Transposons,repeat_features:LTRs,Acceptor splice site,Donor splice site
0,59151,0,0,0,0,0,0,0,0,0,0,0,0,3,1
1,61021,0,0,0,0,0,0,0,0,0,0,0,0,3,1
2,-1,0,0,0,0,0,0,0,0,0,0,0,0,4,4
3,21,0,0,0,0,0,0,0,0,0,0,0,0,3,1
4,526,0,0,0,0,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517420,-1,0,3,5,0,0,1,0,0,0,0,0,0,4,4
517421,-1,37,19,38,7,6,22,1,0,0,0,8,17,4,4
517422,-1,0,0,0,0,0,0,0,0,0,0,0,0,4,4
517423,-1,0,0,0,0,0,0,0,0,0,0,0,0,4,4


In [27]:
# Split the dataset to 9:1 Train/Val and Test set
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier

X_train_val, X_test, y_train_val, y_test = train_test_split(df_annot_test_X_1, df_annot_test_y, stratify=df_annot_test_y, test_size=0.01, shuffle=True)

cv_acc_list = []
cv_ba_acc_list = []
cv_rocauc_list = []
cv_precision_list = []
cv_recall_list = []
cv_mcc_list = []
cv_specificity_list = []
cv_sensitivity_list = []
cv_fpr_list = []
model_dict = {}
model_index = 0
matrix = []

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X_train_val, y_train_val)

for train_index, val_index in skf.split(X_train_val, y_train_val):
	# print(train_index)
	# Perform random under-sampling on the majority class

	print(len(train_index))
	rus = RandomOverSampler(random_state=42, sampling_strategy=.001)
	X_train_resampled, y_train_resampled = rus.fit_resample(X_train_val.iloc[train_index], y_train_val.iloc[train_index])
	print(len(X_train_resampled))

	rus = RandomUnderSampler(random_state=0, sampling_strategy=.8)
	X_train_resampled, y_train_resampled = rus.fit_resample(X_train_resampled, y_train_resampled) #(X_train_resampled, y_train_resampled)  #(X_train_val.iloc[train_index], y_train_val.iloc[train_index])
	print(len(X_train_resampled))

	# Original splits
	# X_train_resampled = X_train_val.iloc[train_index]
	# y_train_resampled = y_train_val.iloc[train_index].to_numpy().flatten()

	# X_train, X_val = X_train_val.iloc[train_index], X_train_val.iloc[val_index]
	# y_train, y_val = y_train_val.iloc[train_index].to_numpy().flatten(), y_train_val.iloc[val_index].to_numpy().flatten()
	X_train, X_val = X_train_resampled, X_train_val.iloc[val_index]
	y_train, y_val = y_train_resampled, y_train_val.iloc[val_index].to_numpy().flatten()

	model_dict[model_index] = XGBClassifier(seed=0)
	model_dict[model_index].fit(X_train, y_train)

	y_predict = model_dict[model_index].predict_proba(X_val)
	y_predict = y_predict[:, 1]
	y_predict_class = list(map(round, y_predict))

	test_acc = accuracy_score(y_val, y_predict_class)
	test_rocauc = roc_auc_score(y_val, y_predict)
	test_bal_acc = balanced_accuracy_score(y_val, y_predict_class)
	test_precision = precision_score(y_val, y_predict_class)  # tp/(tp+fp)
	test_recall = recall_score(y_val, y_predict_class)
	test_mcc = matthews_corrcoef(y_val, y_predict_class)
	matrix.append(confusion_matrix(y_val, y_predict_class))
	tn, fp, fn, tp = confusion_matrix(y_val, y_predict_class).ravel()

	# Adding the metrics to their list
	cv_acc_list.append(test_acc)
	cv_ba_acc_list.append(test_bal_acc)
	cv_rocauc_list.append(test_rocauc)
	cv_precision_list.append(test_precision)
	cv_mcc_list.append(test_mcc)
	cv_recall_list.append(test_recall)
	cv_specificity_list.append(tn / (tn + fp))
	cv_sensitivity_list.append(tp / (fn + tp))
	cv_fpr_list.append(fp/(fp+tn))

409800
409889
920
409800
409890
920
409800
409890
920
409800
409890
920
409800
409890
920


In [29]:
print("------------------------------------------------------------------------------------------")
print("Stratified Cross-Validation Performance")
print("------------------------------------------------------------------------------------------")
print("*Accuracy: %s \nAUCROC: %s \nMCC: %s \nSensitivity: %s \nSpecificity: %s \n*Precision: %s \nRecall: %s \n*False Positive Rate: %s \nBalanced Accuracy: %s" % (
        statistics.mean(cv_acc_list), statistics.mean(cv_rocauc_list), statistics.mean(cv_mcc_list),
        statistics.mean(cv_sensitivity_list), statistics.mean(cv_specificity_list), statistics.mean(cv_precision_list), statistics.mean(cv_recall_list), statistics.mean(cv_fpr_list), statistics.mean(cv_ba_acc_list)))

print("------------------------------------------------------------------------------------------")
print("Accuracy SD: %s \nAUCROC SD: %s \nMCC SD: %s \nSensitivity SD: %s \nSpecificity SD: %s \nPrecision SD: %s  \nBalanced Accuracy SD: %s" % (
        np.std(cv_acc_list), np.std(cv_rocauc_list), np.std(cv_mcc_list),
        np.std(cv_sensitivity_list), np.std(cv_specificity_list), np.std(cv_precision_list), np.std(cv_ba_acc_list)))


print("------------------------------------------------------------------------------------------")

# Print the matrix
(sum(matrix))

------------------------------------------------------------------------------------------
Stratified Cross-Validation Performance
------------------------------------------------------------------------------------------
*Accuracy: 0.8280605173255247 
AUCROC: 0.8703124259650207 
MCC: 0.04285274205418906 
Sensitivity: 0.8281199257162926 
Specificity: 0.7519620253164557 
*Precision: 0.999766660900278 
Recall: 0.8281199257162926 
*False Positive Rate: 0.2480379746835443 
Balanced Accuracy: 0.7900409755163741
------------------------------------------------------------------------------------------
Accuracy SD: 0.0066367363964979905 
AUCROC SD: 0.014253838964525091 
MCC SD: 0.002223983064790129 
Sensitivity SD: 0.006659766452614029 
Specificity SD: 0.036925593281522756 
Precision SD: 3.404173730268859e-05  
Balanced Accuracy SD: 0.016620405752660862
------------------------------------------------------------------------------------------


array([[   300,     99],
       [ 87977, 423874]])

In [10]:
# Next we will evaluate the feature importance of the XGBoost model.
def evaluate_feature_importance(df_evaluate_feature_importance, md):
    feature_importance_average = [np.mean(i) for i in zip(*[md.get(model).feature_importances_ for model in md])]
    df_feature_importance = pd.DataFrame()
    df_feature_importance["Property"] = df_evaluate_feature_importance.columns
    df_feature_importance["Average_importance"] = feature_importance_average
    print(df_feature_importance.sort_values(by="Average_importance", ascending=False).head(50))

evaluate_feature_importance(md=model_dict, df_evaluate_feature_importance=df_annot_test_X_1.copy())

                                   Property  Average_importance
0                                 RC3-Score            0.193397
7                   repeat_features:Unknown            0.115129
2   repeat_features:Type I Transposons/SINE            0.098278
13                     Acceptor splice site            0.096440
6            repeat_features:Tandem repeats            0.079160
12                     repeat_features:LTRs            0.072402
3                      repeat_features:Dust            0.066969
4            repeat_features:Simple repeats            0.065736
1   repeat_features:Type I Transposons/LINE            0.057628
11      repeat_features:Type II Transposons            0.053687
5    repeat_features:Low complexity regions            0.044909
14                        Donor splice site            0.043799
10              repeat_features:RNA repeats            0.012467
8         repeat_features:Satellite repeats            0.000000
9                repeat_features:Centrom